In [4]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()

llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash",
                             temperature=0,
                             api_key=os.getenv("GOOGLE_API_KEY"))


In [6]:
class AgentState(TypedDict):
    message : List[Union[HumanMessage, AIMessage]]
    

In [7]:
def process(state: AgentState) -> AgentState:
    """Process the state by invoking the LLM with the last human message."""
    
    response = llm.invoke(state['message'])
    state['message'].append(AIMessage(content=response.content))
    print(f"AI: {response.content}")
    return state

In [8]:
graph = StateGraph(AgentState)

graph.add_node('process', process)
graph.set_entry_point('process')
graph.set_finish_point('process')
agent = graph.compile()

In [ ]:
convesation_history = []

user_input = input('Enter: ')
while user_input != 'exit':
    convesation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({'message': convesation_history})
    
    print(result['message'])
    convesation_history =  result['message']
    
    user_input =  input('Enter: ')
    
    
    
with open('history.txt', 'w') as file:
    file.write("conversation log\n\n")
    
    for msg in convesation_history:
        if isinstance(msg, HumanMessage):
            file.write(f'You: {msg.content}\n')
        elif isinstance(msg, AIMessage):
            file.write(f'AI: {msg.content}\n\n')
    
    file.write('End of converstion')

print('conversation saved to hisory.txt\n')

AI: Hi there! How can I help you today?
[HumanMessage(content='hi there', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]
AI: Why don't scientists trust atoms?

Because they make up everything!
[HumanMessage(content='hi there', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='tell me a joke', additional_kwargs={}, response_metadata={}), AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={})]
conversation saved to logging.txt
